# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [2]:
import graphlab

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [3]:
def polynomial_sframe(feature, degree):
    poly_sframe = graphlab.SFrame()
    poly_sframe['power_1']=feature
    if degree > 1:
        for power in range(2,degree+1):
            name = 'power_'+str(power)
            poly_sframe[name]=feature**power
    
    return poly_sframe

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

This non-commercial license of GraphLab Create for academic use is assigned to amandalmia18@gmail.com and will expire on August 21, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1474980285.log


In [5]:
sales = graphlab.SFrame('kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [6]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [7]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [9]:
def ridge_regression(data,deg,penalty):
    poly_data=polynomial_sframe(data['sqft_living'],deg)
    poly_data['price']=data['price']
    model = graphlab.linear_regression.create(poly_data,target='price',l2_penalty=penalty,
                                              validation_set=None,verbose=False)
    return model

model = ridge_regression(sales,15,1.5e-5)
print model.get('coefficients')

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   156896.996649    |   932263.046749   |
|   power_1   |  None |   137.412331273    |   4735.67012772   |
|   power_2   |  None |  0.0943456296696   |   9.85922785904   |
|   power_3   |  None |  -0.0001060065121  |  0.0111682653193  |
|   power_4   |  None | 4.50495386925e-08  |  7.6961775401e-06 |
|   power_5   |  None | -6.81277463523e-12 | 3.40377414856e-09 |
|   power_6   |  None | 1.46340653667e-16  | 9.86494005167e-13 |
|   power_7   |  None |  3.8054532936e-20  | 1.85596760052e-16 |
|   power_8   |  None | -2.15430310757e-25 | 2.13681355229e-20 |
|   power_9   |  None | -1.84795223294e-28 | 1.22638795902e-24 |
+-------------+-------+--------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [10]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [11]:
model1=ridge_regression(set_1,15,1.5e-5)
print model1.get('coefficients')

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   1676.88486318    |   1256000.08982   |
|   power_1   |  None |    608.61465132    |   6003.40361437   |
|   power_2   |  None |  -0.421984153102   |   11.9615547364   |
|   power_3   |  None | 0.000153797420248  |  0.0131347719712  |
|   power_4   |  None | -1.81202251743e-08 | 8.84863225398e-06 |
|   power_5   |  None | -1.88380684716e-13 | 3.83743517607e-09 |
|   power_6   |  None | 8.42892999297e-17  | 1.08409739666e-12 |
|   power_7   |  None |  8.1886993898e-21  | 1.93504679159e-16 |
|   power_8   |  None | -5.1488266065e-26  | 1.89388202577e-20 |
|   power_9   |  None | -6.25215798751e-29 |        nan        |
+-------------+-------+--------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows

In [12]:
model2=ridge_regression(set_2,15,1.5e-5)
print model2.get('coefficients')

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   11497.2515406    |   1573134.38921   |
|   power_1   |  None |   673.934728858    |    9294.6775783   |
|   power_2   |  None |  -0.652153371993   |   23.3989079256   |
|   power_3   |  None | 0.000388538138332  |  0.0331234777464  |
|   power_4   |  None | -1.10247234651e-07 |  2.9374688584e-05 |
|   power_5   |  None |  9.4455109742e-12  | 1.72050441742e-08 |
|   power_6   |  None | 1.86449069356e-15  | 6.82534939531e-12 |
|   power_7   |  None | -2.14190170427e-19 | 1.84522758347e-15 |
|   power_8   |  None | -3.65136024098e-23 | 3.35320932115e-19 |
|   power_9   |  None | 5.04195433671e-28  | 3.78737022959e-23 |
+-------------+-------+--------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows

In [13]:
model3=ridge_regression(set_3,15,1.5e-5)
print model3.get('coefficients')

+-------------+-------+--------------------+--------+
|     name    | index |       value        | stderr |
+-------------+-------+--------------------+--------+
| (intercept) |  None |   432563.748301    |  nan   |
|   power_1   |  None |   -681.700050651   |  nan   |
|   power_2   |  None |   0.962655066902   |  nan   |
|   power_3   |  None | -0.000510990370683 |  nan   |
|   power_4   |  None |  1.1962613221e-07  |  nan   |
|   power_5   |  None | -5.20750937245e-12 |  nan   |
|   power_6   |  None | -1.67199818496e-15 |  nan   |
|   power_7   |  None | 5.78939457318e-20  |  nan   |
|   power_8   |  None | 2.11993913316e-23  |  nan   |
|   power_9   |  None | 8.51303190004e-28  |  nan   |
+-------------+-------+--------------------+--------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [16]:
model4=ridge_regression(set_4,15,1.5e-5)
print model4.get('coefficients')

+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   -165030.236873   |   1543874.30104   |
|   power_1   |  None |   1250.93134331    |   9978.92995203   |
|   power_2   |  None |   -1.2655388724    |   26.7151918338   |
|   power_3   |  None | 0.000613088990235  |   0.03920923677   |
|   power_4   |  None | -9.77247684971e-08 | 3.51282080561e-05 |
|   power_5   |  None | -1.32628127198e-11 | 2.00582848476e-08 |
|   power_6   |  None |  4.2929600898e-15  | 7.23485541955e-12 |
|   power_7   |  None | 2.76023952774e-19  | 1.43064913467e-15 |
|   power_8   |  None | -7.09674854411e-23 |        nan        |
|   power_9   |  None | -1.0321375923e-26  |        nan        |
+-------------+-------+--------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [17]:
def print_coefficients(data,deg,penalty):
    model=ridge_regression(data,deg,penalty)
    print model.get('coefficients')

In [19]:
print_coefficients(set_1,15,1e5)

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   530317.024516   |   1890090.19151   |
|   power_1   |  None |   2.58738875673   |   9034.21455077   |
|   power_2   |  None |  0.00127414400592 |    18.000330944   |
|   power_3   |  None | 1.74934226932e-07 |   0.019765845458  |
|   power_4   |  None | 1.06022119097e-11 | 1.33158533722e-05 |
|   power_5   |  None | 5.42247604482e-16 | 5.77475960841e-09 |
|   power_6   |  None | 2.89563828343e-20 | 1.63140263499e-12 |
|   power_7   |  None | 1.65000666351e-24 | 2.91195278612e-16 |
|   power_8   |  None | 9.86081528409e-29 | 2.85000603886e-20 |
|   power_9   |  None | 6.06589348254e-33 |        nan        |
+-------------+-------+-------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, n

In [20]:
print_coefficients(set_2,15,1e5)

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   519216.897383   |   2167962.96521   |
|   power_1   |  None |   2.04470474182   |   12809.1515268   |
|   power_2   |  None |  0.0011314362684  |   32.2464286313   |
|   power_3   |  None | 2.93074277549e-07 |  0.0456480218892  |
|   power_4   |  None | 4.43540598453e-11 |  4.0481752482e-05 |
|   power_5   |  None | 4.80849112204e-15 | 2.37105608016e-08 |
|   power_6   |  None | 4.53091707826e-19 | 9.40612881847e-12 |
|   power_7   |  None | 4.16042910575e-23 | 2.54293917341e-15 |
|   power_8   |  None | 3.90094635128e-27 | 4.62111417353e-19 |
|   power_9   |  None |  3.7773187602e-31 | 5.21943862496e-23 |
+-------------+-------+-------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, n

In [21]:
print_coefficients(set_3,15,1e5)

+-------------+-------+-------------------+--------+
|     name    | index |       value       | stderr |
+-------------+-------+-------------------+--------+
| (intercept) |  None |   522911.518048   |  nan   |
|   power_1   |  None |   2.26890421877   |  nan   |
|   power_2   |  None |  0.00125905041842 |  nan   |
|   power_3   |  None | 2.77552918155e-07 |  nan   |
|   power_4   |  None |  3.2093309779e-11 |  nan   |
|   power_5   |  None | 2.87573572364e-15 |  nan   |
|   power_6   |  None | 2.50076112671e-19 |  nan   |
|   power_7   |  None | 2.24685265906e-23 |  nan   |
|   power_8   |  None | 2.09349983135e-27 |  nan   |
|   power_9   |  None | 2.00435383296e-31 |  nan   |
+-------------+-------+-------------------+--------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [22]:
print_coefficients(set_4,15,1e5)

+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   513667.087087   |   2041482.90233   |
|   power_1   |  None |   1.91040938244   |   13195.2548642   |
|   power_2   |  None |  0.00110058029175 |   35.3258081465   |
|   power_3   |  None | 3.12753987879e-07 |   0.051846828738  |
|   power_4   |  None | 5.50067886825e-11 | 4.64504371162e-05 |
|   power_5   |  None | 7.20467557825e-15 | 2.65233027965e-08 |
|   power_6   |  None | 8.24977249384e-19 | 9.56673326956e-12 |
|   power_7   |  None | 9.06503223498e-23 | 1.89176395105e-15 |
|   power_8   |  None | 9.95683160453e-27 |        nan        |
|   power_9   |  None | 1.10838127982e-30 |        nan        |
+-------------+-------+-------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, n

These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [23]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [24]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [ ]:
train_valid_shuffled[0:10] # rows 0 to 9

Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [32]:
validation4 =train_valid_shuffled[n*3/k : n*4/k]
print len(train_valid_shuffled)

19396


To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [33]:
print int(round(validation4['price'].mean(), 0))

536234


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [ ]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [35]:
first = train_valid_shuffled[0:n*3/k]
last = train_valid_shuffled[n*4/k:n]
train4 = first.append(last)

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [36]:
print int(round(train4['price'].mean(), 0))

539450


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [46]:
def get_residual_sum_of_squares(model,data,outcome):
    predictions = model.predict(data)
    residue = outcome - predictions
    RSS = residue*residue
    return sum(RSS)

In [51]:
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    sum_of_errors = 0
    for i in range(k):
        validation=data[n*i/k : n*(i+1)/k]
        first = data[0:n*i/k]
        second = data[n*(i+1)/k:n]
        training = first.append(second)
        print len(training)
        model = graphlab.linear_regression.create(training,target=output_name,features=features_list,
                                                 l2_penalty=l2_penalty,validation_set=None,verbose=False)
        rss = get_residual_sum_of_squares(model,validation,validation['price'])
        print rss
        sum_of_errors += rss
    
    return sum_of_errors/k
    

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [54]:
import numpy as np
penalty_values=np.logspace(1,7,num=13)
min_error=float('inf')
l2_penalty=0
poly = polynomial_sframe(train_valid_shuffled['sqft_living'],15)
features = poly.column_names()
poly['price']=train_valid_shuffled['price']
for i in range(len(penalty_values)):
    error = k_fold_cross_validation(k,penalty_values[i],poly,'price',features)
    print error
    if(error<min_error):
        min_error = error
        l2_penalty=penalty_values[i]
print l2_penalty

17457
1.09834589206e+14
17456
1.11279322217e+14
17457
9.5736012567e+13
17456
1.22811521665e+14
17456
1.11668727442e+14
17457
3.83515867513e+15
17456
1.35379487989e+14
17457
1.29084332946e+14
17456
1.14341390875e+14
17456
1.52970217654e+14
4.91826427769e+14
17457
1.09858055086e+14
17456
1.11361988868e+14
17457
9.57414318585e+13
17456
1.2273128054e+14
17456
1.11816928759e+14
17457
1.80846254958e+15
17456
1.35361660442e+14
17457
1.29422279682e+14
17456
1.14625991701e+14
17456
1.35660132676e+14
2.87504229919e+14
17457
1.09689854679e+14
17456
1.11402600431e+14
17457
9.59095974929e+13
17456
1.22670965103e+14
17456
1.12047286599e+14
17457
5.47396266561e+14
17456
1.35115248817e+14
17457
1.29889207115e+14
17456
1.14973389774e+14
17456
1.29995241648e+14
1.60908965822e+14
17457
1.09588832704e+14
17456
1.11566247658e+14
17457
9.62092338873e+13
17456
1.22680666046e+14
17456
1.12377678076e+14
17457
1.57324030394e+14
17456
1.34936405311e+14
17457
1.3061570065e+14
17456
1.15366241092e+14
17456
1.30244

***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [ ]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`.  This way, your final model will be trained on the entire dataset.

In [55]:
test

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
8658300340,2014-05-23 00:00:00+00:00,80000.0,1.0,0.75,430.0,5050,1,0
0745000005,2014-08-25 00:00:00+00:00,145000.0,1.0,0.75,480.0,9750,1,0
1623049041,2014-05-08 00:00:00+00:00,82500.0,2.0,1.0,520.0,22334,1,0
6003000851,2014-05-22 00:00:00+00:00,353000.0,1.0,1.0,550.0,1279,2,0
3223039010,2014-08-04 00:00:00+00:00,260000.0,2.0,1.0,570.0,81893,1,0
3163600015,2015-01-28 00:00:00+00:00,156000.0,2.0,1.0,600.0,4000,1,0
1900000035,2015-05-05 00:00:00+00:00,212000.0,1.0,1.0,620.0,7620,1,0
7454001075,2014-06-18 00:00:00+00:00,240000.0,2.0,1.0,670.0,10920,1,0
2113701200,2014-09-12 00:00:00+00:00,250000.0,2.0,1.0,670.0,4640,1,0
2123049114,2014-05-19 00:00:00+00:00,110700.0,2.0,1.0,680.0,8064,1,0


***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***

In [62]:
model=ridge_regression(train_valid,15,1000)
print test['power_15']
print get_residual_sum_of_squares(model,test,test['price'])

[3.1770703657979555e+39, 1.654316344790372e+40, 5.496043412801867e+40, 1.2747949735765537e+41, 2.178325799068597e+41, 4.70184984576e+41, 7.689097049487667e+41, 2.461059085914092e+42, 2.461059085914092e+42, 3.073503348387796e+42, 3.073503348387796e+42, 3.825924196601652e+42, 4.747561509943e+42, 4.747561509943e+42, 5.873205959385493e+42, 5.873205959385493e+42, 7.244150201408991e+42, 7.244150201408991e+42, 7.244150201408991e+42, 7.244150201408991e+42, 7.244150201408991e+42, 7.244150201408991e+42, 7.244150201408991e+42, 8.909286024901501e+42, 1.0926367308245084e+43, 1.0926367308245084e+43, 1.0926367308245084e+43, 1.6300611027433469e+43, 1.6300611027433469e+43, 1.9831742626597601e+43, 1.9831742626597601e+43, 1.9831742626597601e+43, 2.406683831733905e+43, 2.406683831733905e+43, 2.9134419507545593e+43, 2.9134419507545593e+43, 2.9134419507545593e+43, 2.9134419507545593e+43, 2.9134419507545593e+43, 3.5184372088832e+43, 3.5184372088832e+43, 3.5184372088832e+43, 3.5184372088832e+43, 4.23911582752